In [113]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

In [4]:
df=pd.read_csv("data.csv",encoding="ISO-8859-1")

In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [6]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [11]:
# we must convert the CustomerId column to be str (or object) and InvoiceDate to datetime 
df['CustomerID']=df['CustomerID'].astype(str)
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

In [12]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

In [23]:
df.head(8)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom


In [18]:
df[['InvoiceNo','CustomerID']].nunique()

InvoiceNo     25900
CustomerID     4373
dtype: int64

In [30]:
# since for a given invoice number , there's only one customer ..therefore the data seems right on this front
df.groupby('InvoiceNo')['CustomerID'].nunique().max()

1

In [41]:
# there are 43 inovices with 2 different Invoice datetime , let's explore one such case
s=df.groupby('InvoiceNo')['InvoiceDate'].nunique()
len(s[s>1])

43

In [82]:
df['Country'].nunique()

38

In [49]:
print('DataFrame Dimensions :' , df.shape)

DataFrame Dimensions : (541909, 8)


In [64]:
# it's great that we have just one column with null values and that too only 0.26 % , so we can probably drop those rows without any significant loss of data
tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'number of null values'}))
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:' null values %'}))

In [65]:
tab_info

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
number of null values,0,0,1454,0,0,0,0,0
null values %,0,0,0.268311,0,0,0,0,0


In [66]:
df.dropna(axis=0,subset=['Description'],inplace=True)

In [67]:
tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'number of null values'}))
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:' null values %'}))

In [69]:
display(tab_info)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
number of null values,0,0,0,0,0,0,0,0
null values %,0,0,0,0,0,0,0,0


In [78]:
print("Number of duplicates in the dataframe : {} " .format(df.duplicated().sum()))

Number of duplicates in the dataframe : 5268 


In [79]:
df.drop_duplicates(inplace=True)

In [81]:
df.shape

(535187, 8)

In [88]:
# actually the CustomerID column does not have null values but it has value='nan' which is of no use to us , so we need to drop those rows
len(df[df['CustomerID']=='nan'])/df.shape[0]*100

24.96006068906756

In [92]:
df_new=df.loc[df['CustomerID']!='nan']

In [93]:
df.shape

(535187, 8)

In [94]:
df_new.shape

(401604, 8)

In [97]:
df=df_new

In [98]:
temp = df[['CustomerID', 'InvoiceNo', 'Country']].groupby(['CustomerID', 'InvoiceNo', 'Country']).count()
temp = temp.reset_index(drop = False)

In [101]:
countries=temp['Country'].value_counts()

In [133]:
# generating chloropleth maps (using plotly)
data = dict(type='choropleth',locations=countries.index,locationmode = 'country names', z = countries,text=countries.index,
           colorbar = {'title':'# of Orders'},colorscale=[[0, 'rgb(224,255,255)'],
            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],
            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],
            [1, 'rgb(227,26,28)']],    
reversescale = False)

In [140]:
layout = dict(title='Number of orders per country',
geo = dict(showframe = True, projection={'type':'mercator'}))

# We see that the dataset is largely dominated by orders made from the UK.

In [142]:
choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate=False)